# Fine-tune ruBERT-tiny2

Загружаем библиотеки

In [38]:
import pandas as pd
import numpy as np
import torch
import transformers
import nltk
from transformers import AutoModelForMaskedLM, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import load_metric, Dataset
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from io import StringIO

Загружаем отобранные статьи: cтатьи из https://ria.ru/export/rss2/archive/index.xml, которые в течение трех суток были изменены и затем проверены дополнительно через Яндекс Speller

In [53]:
with open('datanews.json', encoding="utf-8") as f:
    read_data = f.read()
read_data = read_data.replace('\n][\n', ',\n')
articles = pd.read_json(StringIO(read_data), orient='records')

Формируем предложения из отобранных статей. Второе предложение типа "МОСКВА, 12 янв – РИА Новости" убираем.
В новый DataFrame записываем предложение и признак, что предложения корректные (правильные)

In [54]:
pst = nltk.PunktSentenceTokenizer()
sentences = pd.DataFrame({"text": [], "label": []})
for ind in articles.index:
    sentArticle = pst.tokenize(articles['Article'][ind])
    m = 0
    for s in sentArticle:
        if m != 1:
            sentences.loc[len(sentences.index)] = [s, 1]
print("Number of correct sentences: ", len(sentences.index))
sentences.head(3)

Number of correct sentences:  4036


,text,label
0,В Госдуме предложили сделать старый Новый год ...,1
1,Председатель союза дачников Подмосковья и депу...,1
2,"""После длинных новогодних праздников людям тяж...",1


Добавляем предложения с ошибками. При этом устанавливаем признак, что предложения неправильные

In [55]:
error_list = []
with open('errorsents.txt', encoding="utf-8") as fe:
    error_list = fe.readlines()
for s in error_list:
    sentences.loc[len(sentences.index)] = [s, 0]
print("Number of incorrect sentences: ", len(error_list))
sentences.tail(3)

Number of incorrect sentences:  55


,text,label
4088,"Комитет ободрил законопроект, передает корресп...",0
4089,Но для ударов по Донецку украинские войска исп...,0
4090,В октябре прошлого года Евтухов в интервью РИА...,0


In [56]:
raw_ds = Dataset.from_pandas(sentences)
raw_ds.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

Datasets:

In [50]:
raw_ds_train = Dataset.from_pandas(train)
raw_ds_test = Dataset.from_pandas(test)
raw_ds_train.features

{'target': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [44]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModelForMaskedLM.from_pretrained("cointegrated/rubert-tiny2", num_labels=2)

In [45]:
def tokenize_function(data):
    return tokenizer(data["text"], truncation=True)
tokenized_ds_train = raw_ds_train.map(tokenize_function, batched=True)
tokenized_ds_test = raw_ds_test.map(tokenize_function, batched=True)

Map:   0%|          | 0/3272 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [46]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [47]:
training_args = TrainingArguments("test-trainer")

In [48]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_ds_train,
    eval_dataset=tokenized_ds_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [49]:
trainer.train()

  0%|          | 0/1227 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.